In [0]:
import math
import torch 
import matplotlib 
from matplotlib import pyplot as plt
from torchvision import transforms, utils
import numpy as np
from numpy.linalg import inv
import torchvision
import torchvision.datasets as datasets
import scipy
from scipy.linalg import *
from scipy.linalg import cholesky, cho_solve, solve
from scipy.special import erf, expit

mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset =datasets.MNIST(root='./data', train=False, download=True, transform=None)
train_x=mnist_trainset.data[0:6000].view(-1,28*28)

def kernel(a, b, param):
    sqdist = (a**2) + (b**2).t() - 2*torch.mm(a, b.t())
    return torch.exp(-.5 * (1/param**2) * sqdist)
sigma=1
param=.1


images= torch.ones([60000,64],dtype=torch.float32)
images_test=torch.ones([10000,64],dtype=torch.float32)

for i in range(60000):
	trans=torchvision.transforms.ToPILImage(mode=None)
	resize = transforms.Resize(size=(8, 8))
	img=resize(trans(mnist_trainset.data[i]))
	tens=torchvision.transforms.ToTensor()
	img=tens(img).view(-1,8*8)
	images[i]= img
for i in range(10000):
	trans=torchvision.transforms.ToPILImage(mode=None)
	resize = transforms.Resize(size=(8, 8))
	img=resize(trans(mnist_testset.data[i]))
	tens=torchvision.transforms.ToTensor()
	img=tens(img).view(-1,8*8)
	images_test[i]= img

0it [00:00, ?it/s]

9920512it [00:01, 8180143.84it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 136924.80it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2247601.35it/s]                            
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 51623.91it/s]            


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
from scipy.spatial.distance import cdist
def kernels(data1,data2,param,ampl):
  dists = cdist(data1 / param, data2 / param,metric='sqeuclidean')
  K = np.exp(-.5 * dists)
  return ampl*K

In [0]:
def matrix_cholesky(A):
    L = torch.zeros_like(A)

    for i in range(A.shape[-1]):
        for j in range(i+1):
            s = 0.0
            for k in range(j):
                s = s + L[i,k] * L[j,k]

            L[i,j] = torch.sqrt(A[i,i] - s) if (i == j) else \
                      (1.0 / L[j,j] * (A[i,j] - s))
    return L

In [0]:
def binar(arr,value):
  arr[np.where(arr!=value)]=-4
  arr[np.where(arr==value)]=-2
  return arr+3

In [0]:
def predict(targets,K,f):
  pi = expit(f)
  f_star = K.T.dot(targets- pi)
  return f_star

In [0]:
def predictprob(targets,K,K_star,K_star_star,f):
  pi=expit(f)
  LAMBDAS = np.array([0.41, 0.4, 0.37, 0.44, 0.39])[:, np.newaxis]
  COEFS = np.array([-1854.8214151, 3516.89893646, 221.29346712,128.12323805, -2010.49422654])[:, np.newaxis]
  f_star = K_star.T.dot(targets - pi)
  W = pi * (1 - pi)
  W_sr = np.sqrt(W)
  W_sr_K = W_sr[:, np.newaxis] * K
  B = np.eye(W.shape[0]) + W_sr_K * W_sr
  L = cholesky(B, lower=True)
  v = solve(L, W_sr[:, np.newaxis] * K_star)  
  var_f_star = np.diag(K_star_star) - np.einsum("ij,ij->j", v, v)
  alpha = 1 / (2 * var_f_star)
  gamma = LAMBDAS * f_star
  integrals = np.sqrt(np.pi / alpha) * erf(gamma * np.sqrt(alpha / (alpha + LAMBDAS**2))) / (2 * np.sqrt(var_f_star * 2 * np.pi))
  pi_star = (COEFS * integrals).sum(axis=0) + .5 * COEFS.sum()
  return np.vstack((pi_star)).T


In [0]:
def newton(targets,K,iteratons):
  f = np.zeros_like(targets, dtype=np.float64)
  log_marginal_likelihood = -np.inf
  for _ in range(iteratons):
            # Line 4
            pi = expit(f)
            W = pi * (1 - pi)
            # Line 5
            W_sr = np.sqrt(W)
            W_sr_K = W_sr[:, np.newaxis] * K
            B = np.eye(W.shape[0]) + W_sr_K * W_sr
            L = cholesky(B, lower=True)
            #L = np.array(matrix_cholesky(torch.as_tensor(B)))
            # Line 6
            b = W * f + (targets - pi)
            # Line 7
            a = b - W_sr * cho_solve((L, True), W_sr_K.dot(b))
            # Line 8
            f = K.dot(a)

            # Line 10: Compute log marginal likelihood in loop and use as
            #          convergence criterion
            lml = -0.5 * a.T.dot(f) \
                - np.log1p(np.exp(-(targets * 2 - 1) * f)).sum() \
                - np.log(np.diag(L)).sum()
            # Check if we have converged (log marginal likelihood does
            # not decrease)
            # XXX: more complex convergence criterion
            if lml - log_marginal_likelihood < 1e-10:
                break
            log_marginal_likelihood = lml
            #print(lml)
  return f

In [0]:
samples=2000
lscale=[9678.83555037,2485.89159882,2787.85417398,3176.2763532,3965.89015053,2201.46584303,4394.15303759,5227.57692151,2798.70785485,2132.15866156]
ampl=[2170.53098164,226.27154633,162.94049999,219.6163741,300.88828563,105.99065007,408.15960278,776.39586001,142.06318177,97.67561165]
testsamples=100
maxima = np.empty(testsamples, dtype=float)
maxima.fill(-np.inf)
argmaxima = np.zeros(testsamples, dtype=int)
maximaprob = np.empty(testsamples, dtype=float)
maximaprob.fill(-np.inf)
for i in range(10):
  targets=binar(np.array(mnist_trainset.targets[0:samples]),i)
  kernel=kernels(np.array(train_x[0:samples]),np.array(train_x[0:samples]),lscale[i],ampl[i])
  f2=newton((targets+1)/2,kernel,100)
  ks=kernels(np.array(train_x[0:samples]),np.array(train_x[samples:samples+testsamples]),lscale[i],ampl[i])
  star=predict((targets+1)/2,ks,f2)
  K_star_star=kernels(np.array(train_x[samples:samples+testsamples]),np.array(train_x[samples:samples+testsamples]),lscale[i],ampl[i])
  prob=predictprob((targets+1)/2,kernel,ks,K_star_star,f2)
  np.maximum(maxima, star, out=maxima)
  argmaxima[maxima == star] = i
  np.maximum(maximaprob[np.newaxis,:], prob, out=maximaprob[np.newaxis,:])
  #print(star)
print(mnist_trainset.targets[samples:samples+testsamples])
print(argmaxima)
print(maximaprob)

tensor([5, 8, 8, 4, 2, 6, 9, 7, 1, 0, 2, 3, 1, 8, 7, 1, 8, 2, 7, 2, 6, 3, 2, 3,
        7, 6, 8, 7, 5, 3, 8, 4, 4, 9, 4, 6, 4, 5, 5, 1, 1, 9, 3, 8, 6, 1, 4, 1,
        7, 5, 4, 0, 4, 2, 7, 7, 5, 8, 0, 6, 9, 6, 6, 8, 7, 2, 0, 8, 8, 4, 8, 4,
        4, 2, 3, 2, 3, 8, 2, 0, 5, 0, 0, 1, 0, 2, 1, 3, 3, 4, 7, 5, 3, 6, 1, 7,
        6, 8, 3, 9])
[5 8 8 4 2 6 9 7 1 0 2 3 1 8 0 1 8 2 7 2 6 3 2 3 7 6 8 7 5 3 8 4 4 9 2 6 4
 5 5 1 1 9 3 8 1 1 4 1 7 5 4 0 4 2 7 7 5 8 0 6 9 6 6 8 2 2 0 8 8 4 1 4 4 2
 3 2 3 8 2 0 5 0 0 1 0 2 1 3 3 4 7 5 4 6 1 7 6 8 7 9]
[0.37218178 0.94903751 0.64886016 0.88840215 0.91092029 0.96055918
 0.87146247 0.99201279 0.91090209 0.98363645 0.46147711 0.82357475
 0.81164542 0.90137263 0.57666956 0.90167279 0.54528378 0.88209292
 0.65100363 0.857855   0.8404419  0.89002231 0.71546678 0.60587859
 0.94893599 0.9353431  0.58891535 0.96916035 0.48159478 0.72579318
 0.34580877 0.96926259 0.77891661 0.84117858 0.39656423 0.91951977
 0.50656145 0.8461704  0.82590783 0.77460945 0.551341

In [0]:
accuracy = (np.array(mnist_trainset.targets[samples:samples+testsamples]) == argmaxima).mean()
print(accuracy)

0.93
